In [1]:
import numpy as np

import pickle
from feeder import tools

import torch
import torch.nn as nn

from net import st_gcn

In [2]:
class Feeder(torch.utils.data.Dataset):
    """ Feeder for skeleton-based action recognition
    Arguments:
        data_path: the path to '.npy' data, the shape of data should be (N, C, T, V, M)
        label_path: the path to label
        random_choose: If true, randomly choose a portion of the input sequence
        random_shift: If true, randomly pad zeros at the begining or end of sequence
        window_size: The length of the output sequence
        normalization: If true, normalize input sequence
        debug: If true, only use the first 100 samples
    """

    def __init__(self,
                 data_path,
                 label_path,
                 random_choose=False,
                 random_move=False,
                 window_size=-1,
                 debug=False,
                 mmap=True):
        self.debug = debug
        self.data_path = data_path
        self.label_path = label_path
        self.random_choose = random_choose
        self.random_move = random_move
        self.window_size = window_size

        self.load_data(mmap)

    def load_data(self, mmap):
        # data: N C V T M

        # load label
        with open(self.label_path, 'rb') as f:
            self.sample_name, self.label = pickle.load(f)

        # load data
        if mmap:
            self.data = np.load(self.data_path, mmap_mode='r')
        else:
            self.data = np.load(self.data_path)

        if self.debug:
            self.label = self.label[0:100]
            self.data = self.data[0:100]
            self.sample_name = self.sample_name[0:100]

        self.N, self.C, self.T, self.V, self.M = self.data.shape

    def __len__(self):
        return len(self.label)

    def __getitem__(self, index):
        # get data
        data_numpy = np.array(self.data[index])
        label = self.label[index]

        # processing
        if self.random_choose:
            data_numpy = tools.random_choose(data_numpy, self.window_size)
        elif self.window_size > 0:
            data_numpy = tools.auto_pading(data_numpy, self.window_size)
        if self.random_move:
            data_numpy = tools.random_move(data_numpy)

        return data_numpy, label

In [3]:
label_name_path = './resource/kinetics_skeleton/label_name.txt'
with open(label_name_path) as f:
    label_name = f.readlines()
    label_name = [line.rstrip() for line in label_name]

In [4]:
dev = "cuda:0"

label_name_path = './resource/kinetics_skeleton/label_name.txt'
with open(label_name_path) as f:
    label_name = f.readlines()
    label_name = [line.rstrip() for line in label_name]

data_path = "./data/NTU-RGB-D/xsub/val_data.npy"
label_path = "./data/NTU-RGB-D/xsub/val_label.pkl"
dset = Feeder(data_path, label_path)

data_loader = torch.utils.data.DataLoader(
    dataset=dset,
    batch_size=64,
    shuffle=False)

graph_args = {'layout': 'ntu-rgb+d', 'strategy': 'spatial'}

model = st_gcn.Model(in_channels=3, num_class=60,
                     graph_args=graph_args, edge_importance_weighting=True)
weights = torch.load("./work_dir/recognition/ntu-xsub/ST_GCN/epoch50_model.pt")
model.load_state_dict(weights)
model.eval()

model = model.to(dev)

loss_value = []
result_frag = []
label_frag = []

In [5]:
output_all = []
voting_label_all = []
label_all = []

In [6]:
for data, label in data_loader:
    data = data.float().to(dev)
    label = label.long().to(dev)

    with torch.no_grad():
        output_batch, feature_batch = model.extract_feature(data)     

        for i in range (len(label)):
            output = output_batch[i]
            feature = feature_batch[i]
            intensity = (feature * feature).sum(dim=0) ** 0.5
            intensity = intensity.cpu().detach().numpy()

            # classification result of the full sequence
            voting_label = output.sum(dim=3).sum(
                dim=2).sum(dim=1).argmax(dim=0)
            voting_label_name = label_name[voting_label]
            print("voting_label_name is ", voting_label_name)

            real_label_name = label_name[label[i]]
            print("real label is", real_label_name)

            output_all.append(output.data.cpu().numpy())
            voting_label_all.append(voting_label.data.cpu().numpy())
            label_all.append(label[i].data.cpu().numpy())

voting_label_name is  blowing leaves
real label is blowing leaves
voting_label_name is  clay pottery making
real label is clay pottery making
voting_label_name is  bench pressing
real label is bench pressing
voting_label_name is  bench pressing
real label is bench pressing
voting_label_name is  clapping
real label is changing oil
voting_label_name is  brush painting
real label is brush painting
voting_label_name is  carrying baby
real label is blowing glass
voting_label_name is  building cabinet
real label is building shed
voting_label_name is  bending metal
real label is bending metal
voting_label_name is  baby waking up
real label is bobsledding
voting_label_name is  archery
real label is archery
voting_label_name is  catching or throwing frisbee
real label is catching or throwing frisbee
voting_label_name is  clapping
real label is clapping
voting_label_name is  bending back
real label is bending back
voting_label_name is  checking tires
real label is checking tires
voting_label_nam

voting_label_name is  auctioning
real label is applauding
voting_label_name is  checking tires
real label is chopping wood
voting_label_name is  brush painting
real label is brush painting
voting_label_name is  archery
real label is archery
voting_label_name is  blowing out candles
real label is air drumming
voting_label_name is  clapping
real label is clapping
voting_label_name is  clay pottery making
real label is clay pottery making
voting_label_name is  canoeing or kayaking
real label is canoeing or kayaking
voting_label_name is  brushing hair
real label is brushing hair
voting_label_name is  bartending
real label is breakdancing
voting_label_name is  beatboxing
real label is beatboxing
voting_label_name is  brush painting
real label is brush painting
voting_label_name is  building shed
real label is building shed
voting_label_name is  breading or breadcrumbing
real label is auctioning
voting_label_name is  abseiling
real label is abseiling
voting_label_name is  clean and jerk
real

voting_label_name is  catching or throwing baseball
real label is catching or throwing baseball
voting_label_name is  blowing leaves
real label is blowing leaves
voting_label_name is  abseiling
real label is abseiling
voting_label_name is  balloon blowing
real label is balloon blowing
voting_label_name is  arranging flowers
real label is arranging flowers
voting_label_name is  blasting sand
real label is blasting sand
voting_label_name is  canoeing or kayaking
real label is canoeing or kayaking
voting_label_name is  archery
real label is archery
voting_label_name is  capoeira
real label is carving pumpkin
voting_label_name is  bending metal
real label is bending metal
voting_label_name is  balloon blowing
real label is balloon blowing
voting_label_name is  building cabinet
real label is building cabinet
voting_label_name is  belly dancing
real label is belly dancing
voting_label_name is  building cabinet
real label is building cabinet
voting_label_name is  blowing nose
real label is bl

voting_label_name is  carrying baby
real label is carrying baby
voting_label_name is  abseiling
real label is bowling
voting_label_name is  cartwheeling
real label is abseiling
voting_label_name is  bungee jumping
real label is building cabinet
voting_label_name is  air drumming
real label is air drumming
voting_label_name is  changing oil
real label is changing oil
voting_label_name is  blowing leaves
real label is blowing leaves
voting_label_name is  cartwheeling
real label is cartwheeling
voting_label_name is  bowling
real label is bouncing on trampoline
voting_label_name is  bee keeping
real label is blowing glass
voting_label_name is  busking
real label is busking
voting_label_name is  arranging flowers
real label is arranging flowers
voting_label_name is  bee keeping
real label is bee keeping
voting_label_name is  bouncing on trampoline
real label is bouncing on trampoline
voting_label_name is  balloon blowing
real label is balloon blowing
voting_label_name is  blasting sand
real

voting_label_name is  breading or breadcrumbing
real label is breading or breadcrumbing
voting_label_name is  assembling computer
real label is assembling computer
voting_label_name is  capoeira
real label is capoeira
voting_label_name is  bungee jumping
real label is breakdancing
voting_label_name is  brushing hair
real label is brushing hair
voting_label_name is  applauding
real label is catching or throwing baseball
voting_label_name is  auctioning
real label is changing oil
voting_label_name is  carrying baby
real label is cartwheeling
voting_label_name is  bee keeping
real label is brushing hair
voting_label_name is  celebrating
real label is celebrating
voting_label_name is  building cabinet
real label is building cabinet
voting_label_name is  blowing out candles
real label is blowing out candles
voting_label_name is  applying cream
real label is applying cream
voting_label_name is  baby waking up
real label is baby waking up
voting_label_name is  cheerleading
real label is cheer

...

voting_label_name is  brushing hair
real label is brushing hair
voting_label_name is  clapping
real label is clapping
voting_label_name is  air drumming
real label is air drumming
voting_label_name is  cartwheeling
real label is chopping wood
voting_label_name is  answering questions
real label is answering questions
voting_label_name is  barbequing
real label is barbequing
voting_label_name is  clean and jerk
real label is clean and jerk
voting_label_name is  celebrating
real label is celebrating
voting_label_name is  bending metal
real label is bending metal
voting_label_name is  breading or breadcrumbing
real label is auctioning
voting_label_name is  bending metal
real label is bending metal
voting_label_name is  checking tires
real label is checking tires
voting_label_name is  changing oil
real label is changing wheel
voting_label_name is  balloon blowing
real label is balloon blowing
voting_label_name is  brush painting
real label is brush painting
voting_label_name is  canoeing o

voting_label_name is  breading or breadcrumbing
real label is breading or breadcrumbing
voting_label_name is  auctioning
real label is auctioning
voting_label_name is  breading or breadcrumbing
real label is breading or breadcrumbing
voting_label_name is  arm wrestling
real label is arm wrestling
voting_label_name is  capoeira
real label is applauding
voting_label_name is  clean and jerk
real label is clean and jerk
voting_label_name is  celebrating
real label is celebrating
voting_label_name is  bartending
real label is bartending
voting_label_name is  bee keeping
real label is bee keeping
voting_label_name is  bouncing on trampoline
real label is bouncing on trampoline
voting_label_name is  bartending
real label is bartending
voting_label_name is  carrying baby
real label is carrying baby
voting_label_name is  barbequing
real label is barbequing
voting_label_name is  carrying baby
real label is cartwheeling
voting_label_name is  blowing glass
real label is blowing glass
voting_label_

voting_label_name is  breakdancing
real label is breakdancing
voting_label_name is  abseiling
real label is abseiling
voting_label_name is  catching or throwing softball
real label is catching or throwing softball
voting_label_name is  catching or throwing frisbee
real label is catching or throwing frisbee
voting_label_name is  breakdancing
real label is breakdancing
voting_label_name is  beatboxing
real label is beatboxing
voting_label_name is  breakdancing
real label is breakdancing
voting_label_name is  catching fish
real label is catching fish
voting_label_name is  blowing nose
real label is blowing nose
voting_label_name is  bungee jumping
real label is bungee jumping
voting_label_name is  catching or throwing frisbee
real label is catching or throwing frisbee
voting_label_name is  brushing hair
real label is brushing hair
voting_label_name is  bobsledding
real label is bobsledding
voting_label_name is  changing wheel
real label is changing wheel
voting_label_name is  blowing glas

voting_label_name is  carrying baby
real label is breading or breadcrumbing
voting_label_name is  building cabinet
real label is building cabinet
voting_label_name is  bee keeping
real label is bee keeping
voting_label_name is  bartending
real label is archery
voting_label_name is  brushing hair
real label is brushing hair
voting_label_name is  bending metal
real label is bending metal
voting_label_name is  blowing glass
real label is blowing glass
voting_label_name is  balloon blowing
real label is balloon blowing
voting_label_name is  brushing teeth
real label is brushing teeth
voting_label_name is  building cabinet
real label is building cabinet
voting_label_name is  balloon blowing
real label is balloon blowing
voting_label_name is  abseiling
real label is bowling
voting_label_name is  bench pressing
real label is bench pressing
voting_label_name is  beatboxing
real label is beatboxing
voting_label_name is  arm wrestling
real label is arm wrestling
voting_label_name is  carrying ba

voting_label_name is  busking
real label is busking
voting_label_name is  clean and jerk
real label is clean and jerk
voting_label_name is  carving pumpkin
real label is carving pumpkin
voting_label_name is  clapping
real label is clapping
voting_label_name is  brushing hair
real label is brushing hair
voting_label_name is  biking through snow
real label is bending metal
voting_label_name is  bench pressing
real label is bench pressing
voting_label_name is  applying cream
real label is applying cream
voting_label_name is  carrying baby
real label is carrying baby
voting_label_name is  bookbinding
real label is bookbinding
voting_label_name is  chopping wood
real label is chopping wood
voting_label_name is  catching or throwing frisbee
real label is catching or throwing frisbee
voting_label_name is  biking through snow
real label is biking through snow
voting_label_name is  carving pumpkin
real label is carving pumpkin
voting_label_name is  brushing teeth
real label is brushing teeth
vo

In [7]:
np.shape(output_all)

(16487, 60, 75, 25, 2)

In [8]:
np.shape(label_all)

(16487,)

In [9]:
def resize(x):
    N, C, T, V, M = x.size()
    x = x.permute(0, 4, 3, 1, 2).contiguous()
    x = x.view(N * M, V * C * T)
    return x

In [10]:
tsne_data = resize(torch.tensor(output_all))
tsne_data.shape

torch.Size([32974, 112500])

In [11]:
data = tsne_data[::2,]
label = np.array(label_all)
print(data.shape)
print(label.shape)

torch.Size([16487, 112500])
(16487,)


In [12]:
from time import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.manifold import TSNE

def plot_embedding_2D(data, label, title):
    x_min, x_max = np.min(data, 0), np.max(data, 0)
    data = (data - x_min) / (x_max - x_min)
    fig = plt.figure()
    for i in range(data.shape[0]):
        plt.text(data[i, 0], data[i, 1], str(label[i]),
                 color=plt.cm.Set1(label[i]),
                 fontdict={'weight': 'bold', 'size': 9})
    plt.xticks([])
    plt.yticks([])
    plt.title(title)
    return fig
def plot_embedding_3D(data,label,title): 
    x_min, x_max = np.min(data,axis=0), np.max(data,axis=0) 
    data = (data- x_min) / (x_max - x_min) 
    fig = plt.figure()
    ax = fig.add_subplot(111,projection='3d') 
    for i in range(data.shape[0]): 
        ax.text(data[i, 0], data[i, 1], data[i,2],str(label[i]), color=plt.cm.Set1(label[i]),fontdict={'weight': 'bold', 'size': 9}) 
    return fig

In [ ]:
print('Begining......') #时间会较长，所有处理完毕后给出finished提示
tsne_2D = TSNE(n_components=2, init='pca', random_state=0) #调用TSNE
result_2D = tsne_2D.fit_transform(data)
tsne_3D = TSNE(n_components=3, init='pca', random_state=0)
result_3D = tsne_3D.fit_transform(data)
print('Finished......')
#调用上面的两个函数进行可视化
fig1 = plot_embedding_2D(result_2D, label,'t-SNE')
plt.show(fig1)
fig2 = plot_embedding_3D(result_3D, label,'t-SNE')
plt.show(fig2)

Begining......
